In [15]:
import os
import pandas as pd
import plotly.express as px
from lib.env import *
def run_new_state_machine_on_thresholded_predictions(predictions):
    state = 0
    states = []
    puff_locations = []
    currentInterPuffIntervalLength = 0
    currentPuffLength = 0
    for i,smokingOutput in enumerate(predictions):
        states.append(state)
        if (state == 0 and smokingOutput == 0.0):
            # no action
            state = 0
        elif (state == 0 and smokingOutput == 1.0):
            # starting validating puff length
            state = 1
            currentPuffLength += 1
        elif (state == 1 and smokingOutput == 1.0):
            # continuing not yet valid length puff
            currentPuffLength += 1
            if (currentPuffLength > 14) :
                # valid puff length!
                state = 2
        elif (state == 1 and smokingOutput == 0.0):
            # never was a puff, begin validating end
            state = 3
            currentInterPuffIntervalLength += 1
        elif (state == 2 and smokingOutput == 1.0):
            # continuing already valid puff
            currentPuffLength += 1
        elif (state == 2 and smokingOutput == 0.0):
            # ending already valid puff length
            state = 4 # begin validating inter puff interval
            currentInterPuffIntervalLength += 1
        elif (state == 3 and smokingOutput == 0.0): 
            currentInterPuffIntervalLength += 1
            if (currentInterPuffIntervalLength > 49):
                # valid interpuff
                state = 0
                currentPuffLength = 0
                currentInterPuffIntervalLength = 0
        elif (state == 3 and smokingOutput == 1.0):
            # was validating interpuff for puff that wasn't valid
            currentPuffLength += 1
            currentInterPuffIntervalLength = 0
            if (currentPuffLength > 14) :
                # valid puff length!
                state = 2
            else:
                state = 1
        elif (state == 4 and smokingOutput == 0.0) :
            currentInterPuffIntervalLength += 1
            if (currentInterPuffIntervalLength > 49):
                # valid interpuff for valid puff
                state = 0
                currentPuffLength = 0
                currentInterPuffIntervalLength = 0
                puff_locations.append(i)
        elif (state == 4 and smokingOutput == 1.0):
            # back into puff for already valid puff
            currentInterPuffIntervalLength = 0
            currentPuffLength += 1
            state = 2
    states = states[1:] + [0]
    return states,puff_locations
DATASET_PATH = f'{DATA_PATH}/thrasher'

In [16]:
RECORDING_IDS = os.listdir(DATASET_PATH)
RECORDING_PATH = f'{DATASET_PATH}/{RECORDING_IDS[0]}/raw/{RECORDING_IDS[0]}.0.csv'
df = pd.read_csv(RECORDING_PATH,skiprows=1)
df = df.reset_index()
fs = 20
df.timestamp = (df.timestamp - df.timestamp[0])*1e-9
df['index'] = df['index']/(fs*60) # index in minutes

In [17]:
import torch
from torch import nn
X = torch.from_numpy(df[['acc_x','acc_y','acc_z']].to_numpy())
x = X[:,0].unsqueeze(1)
y = X[:,1].unsqueeze(1)
z = X[:,2].unsqueeze(1)
xs = [x[:-99]]
ys = [y[:-99]]
zs = [z[:-99]]
for i in range(1,99):
    xs.append(x[i:i-99])
    ys.append(y[i:i-99])
    zs.append(z[i:i-99])
xs.append(x[99:])
ys.append(y[99:])
zs.append(z[99:])
xs = torch.cat(xs,axis=1).float()
ys = torch.cat(ys,axis=1).float()
zs = torch.cat(zs,axis=1).float()
X = torch.cat([xs,ys,zs],axis=1)

In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm

class Casey1p1():
    def __init__(self):
        self.range = pd.read_csv(f'{DATA_PATH}/casey_network/range',header=None).to_numpy()
        self.fc1 = pd.read_csv(f'{DATA_PATH}/casey_network/input',header=None).to_numpy()
        self.fc2 = pd.read_csv(f'{DATA_PATH}/casey_network/hidden',header=None).to_numpy()
    def __minMaxNorm(self,x):
        return ((2*((x-self.range[:,0])/(self.range[:,1]-self.range[:,0])))-1).tolist()
    def __tanSigmoid(self,x):
        output = []
        for xi in x:
            output.append((2/(1+np.exp(-2*xi)))-1)
        return output
    def __logSigmoid(self,x):
        return (1/(1+np.exp(-1*x)))
    def __forwardSingleBatch(self,x):
        x = [1] + self.__minMaxNorm(x)
        x = [1] + self.__tanSigmoid(self.fc1 @ x)
        x = self.fc2 @ x
        x = self.__logSigmoid(x[0])
        return x
    def __call__(self,x):
        output = []
        for xi in tqdm(x):
            output.append(self.__forwardSingleBatch(xi))
        return output + [0]*99
model = Casey1p1()

In [19]:
y_pred = model(X)
df['y_pred'] = y_pred

100%|██████████| 133204/133204 [00:09<00:00, 13986.76it/s]


In [20]:
# see the difference in activation
fig = px.line(df.iloc[::2],y=['rawlabel','y_pred'])
fig.show(renderer='browser')

In [21]:
y_thresh = [1 if y > .85 else 0 for y in y_pred]
df['y_thresh'] = y_thresh

In [22]:
fig = px.line(df.iloc[::2],y=['label','y_thresh'])
fig.show(renderer='browser')

In [23]:
y_state,puff_locations = run_new_state_machine_on_thresholded_predictions(y_thresh)

In [24]:
df['y_state'] = y_state
fig = px.line(df.iloc[::2],y=['state','y_state'])
fig.show(renderer='browser')

In [25]:
fig = px.line(df.iloc[::2],y=['state','y_state'])
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc)
fig.show(renderer='browser')